# Data Scientist - P7 - Laurent Trichet

## Implémentez un modèle de scoring

## 2 Feature Engineering - Fill nan, joining data

### Import required libraries - Constants

#### Execute this notebook with NUMROWS=15000 (Classification optimization), and NUMROWS=1000000 (>360000 = full dataset) for API Model creation

In [1]:
# Import default libraries
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Garbage Collector (empty dataFrame memory)
import gc

# tools for execution time estimates
from datetime import datetime

# Remove some warnings
import warnings
warnings.filterwarnings('ignore')

# Pandas parameters
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 120)
pd.set_option('display.max_info_rows', 2000)

# Matplotlib and sns visual parameters
# sns.set_palette("Set2")
sns.set_style('whitegrid')
sns.set_context('paper')
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['xtick.labelsize'] = 11
mpl.rcParams['ytick.labelsize'] = 11

# Constants
DIRSOURCE = '../Sources/'
DIRRESULT = './credithome_datasets/'
NUMROWS = 15000    # 1000000 to get complete dateset
# File names with NUMROWS lines and Fill nan with zeros
FILESTD_FNAN0 = DIRRESULT+'Credit_Home_Junction_Std_Fnan0_'+str(NUMROWS)+'.csv'
FILELOG_FNAN0 = DIRRESULT+'Credit_Home_Junction_Log_Fnan0_'+str(NUMROWS)+'.csv'


### 2.1 Functions files reading, grouping information before joints

In [2]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns \
                               if df[col].dtype == 'object'
                          ]
    df = pd.get_dummies(df,
                        columns= categorical_columns,
                        dummy_na=nan_as_category
                        )
    new_columns = [c for c in df.columns \
                     if c not in original_columns
                  ]
    return df, new_columns


In [3]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows=None, nan_as_category=False):
    # Read data and merge
    df = pd.read_csv(DIRSOURCE+'application_train.csv', nrows=num_rows,
                     encoding='Latin-1')
    test_df = pd.read_csv(DIRSOURCE+'application_test.csv', nrows=num_rows,
                     encoding='Latin-1')
    df = df.append(test_df).reset_index()
     
    # Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Remove 1 application with -1 REGION_RATING_CLIENT_W_CITY (test set)
    df = df[df['REGION_RATING_CLIENT_W_CITY'] != -1]
    
    # Remove 1 application with 0 FLAG_MOBILE (test set).
    # Remove column FLAG_MOBIL (always = 1)
    df = df[df['FLAG_MOBIL'] == 1]
    df = df.drop(['FLAG_MOBIL'], axis=1)
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category=nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    # Replace days birth with years birth
    df['DAYS_BIRTH'] = df['DAYS_BIRTH'] // 365 
    
    # Some simple new features (percentages)
    # REMOVED, RISK OF DIVISOR = 0
    # df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    # df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    # df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    # df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    # df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    del test_df
    gc.collect()
    return df


In [4]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(nan_as_category=True):
    bureau = pd.read_csv(DIRSOURCE+'bureau.csv', encoding='Latin-1')
    bb = pd.read_csv(DIRSOURCE+'bureau_balance.csv', encoding='Latin-1')
    
    bb, bb_cat = one_hot_encoder(bb,
                                 nan_as_category=nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau,
                                         nan_as_category=nan_as_category)
    
    # Bureau balance: Perform aggregations & merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU')\
               .agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() \
                              for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR')\
                       .agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() \
                                  for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() \
                                  for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() \
                                  for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg


In [5]:
# Preprocess previous_applications.csv
def previous_applications(nan_as_category=True, col_log=False):
    if not col_log:
        prev = pd.read_csv(DIRSOURCE+'previous_application.csv',
                           encoding='Latin-1')
    else:
        prev = pd.read_csv(DIRSOURCE+'previous_application_log.csv',
                           encoding='Latin-1', sep='\t')

    prev, cat_cols = one_hot_encoder(prev,
                                     nan_as_category=nan_as_category)
    
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    # Add feature: value ask / value received percentage
    # REMOVED, RISK OF DIVISOR = 0
    # prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        # 'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR')\
                   .agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() \
                                for e in prev_agg.columns.tolist()])
    
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() \
                                    for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() \
                                   for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg


In [6]:
# Preprocess POS_CASH_balance.csv
def pos_cash(nan_as_category=True):
    pos = pd.read_csv(DIRSOURCE+'POS_CASH_balance.csv',
                      encoding='Latin-1')
    
    pos, cat_cols = one_hot_encoder(pos,
                                    nan_as_category=nan_as_category)
    
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() \
                               for e in pos_agg.columns.tolist()])
    
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg


In [7]:
# Preprocess installments_payments.csv
def installments_payments(nan_as_category=True, col_log=False):
    if not col_log:
        ins = pd.read_csv(DIRSOURCE+'installments_payments.csv',
                          encoding='Latin-1')
    else:
        ins = pd.read_csv(DIRSOURCE+'installments_payments_log.csv',
                          encoding='Latin-1', sep='\t')

    ins, cat_cols = one_hot_encoder(ins,
                                    nan_as_category=nan_as_category)
    
    # Percent & diff paid in each installment
    # (amount paid & installment value)
    # ISSUE WITH PERC, SOME DIVISOR = 0
    # ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        # 'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() \
                               for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg


In [8]:
# Preprocess credit_card_balance.csv
def credit_card_balance(nan_as_category=True, col_log=False):
    if not col_log:
        cc = pd.read_csv(DIRSOURCE+'credit_card_balance.csv',
                         encoding='Latin-1')
    else:
        cc = pd.read_csv(DIRSOURCE+'credit_card_balance_log.csv',
                         encoding='Latin-1', sep='\t')

    cc, cat_cols = one_hot_encoder(cc,
                                   nan_as_category=nan_as_category)
    
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR')\
               .agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() \
                              for e in cc_agg.columns.tolist()])
    
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg


### 2.2 Grouping and joining files - With standard Data (no log transformation)

> joints require to read the complete files out of "application_train/test" in order to retrieve all lines with keys in selected rows (NUMROWS)

In [9]:
d1 = datetime.now()

df = application_train_test(num_rows=NUMROWS)
print(f'Concat train samples: {NUMROWS}, test samples: {NUMROWS}, shape: {df.shape}')

bureau = bureau_and_balance()
print(f'Join Bureau & Bureau_balance SK_ID_BUREAU, group by SK_ID_CURR, shape: {bureau.shape}')
df = df.join(bureau, how='left', on='SK_ID_CURR')
del bureau
gc.collect()

prev = previous_applications()
print(f'Group Previous applications by SK_ID_CURR, shape: {prev.shape}')
df = df.join(prev, how='left', on='SK_ID_CURR')
del prev
gc.collect()

pos = pos_cash()
print(f'Group Pos-cash balance by SK_ID_CURR, shape: {pos.shape}')
df = df.join(pos, how='left', on='SK_ID_CURR')
del pos
gc.collect()

ins = installments_payments()
print(f'Group Installments payments by SK_ID_CURR, shape: {ins.shape}')
df = df.join(ins, how='left', on='SK_ID_CURR')
del ins
gc.collect()

cc = credit_card_balance()
print(f'Group Credit card balance by SK_ID_CURR, shape: {cc.shape}')
df = df.join(cc, how='left', on='SK_ID_CURR')
del cc
gc.collect()

print(f'After {datetime.now()-d1}, join results in Application, shape: {df.shape}')


Concat train samples: 15000, test samples: 15000, shape: (30000, 240)
Join Bureau & Bureau_balance SK_ID_BUREAU, group by SK_ID_CURR, shape: (305811, 116)
Group Previous applications by SK_ID_CURR, shape: (338857, 237)
Group Pos-cash balance by SK_ID_CURR, shape: (337252, 18)
Group Installments payments by SK_ID_CURR, shape: (339587, 22)
Group Credit card balance by SK_ID_CURR, shape: (103558, 141)
After 0:01:53.118157, join results in Application, shape: (30000, 774)


#### Reverse target, Set to 1 positive loans, 0 with payment difficulties 
#### Set target of test set to 999
#### Fill other NaNs with 0 (could be completed with alternate methods); Save to disk

In [10]:
df['TARGET'] = 1 - df['TARGET']
df['TARGET'].fillna(999, inplace=True)
df.fillna(0, inplace=True)

df.to_csv(FILESTD_FNAN0, index=None, sep='\t')
print(f'File saved {FILESTD_FNAN0}, shape: {df.shape}')

del df
gc.collect()


File saved ./credithome_datasets/Credit_Home_Junction_Std_Fnan0_15000.csv, shape: (30000, 774)


16

### 2.3 Grouping and joining files - With log Data (log transformation of some columns)

In [11]:
d1 = datetime.now()

df = application_train_test(num_rows=NUMROWS)
print(f'Concat train samples: {NUMROWS}, test samples: {NUMROWS}, shape: {df.shape}')

bureau = bureau_and_balance()
print(f'Join Bureau & Bureau_balance SK_ID_BUREAU, group by SK_ID_CURR, shape: {bureau.shape}')
df = df.join(bureau, how='left', on='SK_ID_CURR')
del bureau
gc.collect()

prev = previous_applications(col_log=True)
print(f'Group Previous applications (log) by SK_ID_CURR, shape: {prev.shape}')
df = df.join(prev, how='left', on='SK_ID_CURR')
del prev
gc.collect()

pos = pos_cash()
print(f'Group Pos-cash balance by SK_ID_CURR, shape: {pos.shape}')
df = df.join(pos, how='left', on='SK_ID_CURR')
del pos
gc.collect()

ins = installments_payments(col_log=True)
print(f'Group Installments payments (log) by SK_ID_CURR, shape: {ins.shape}')
df = df.join(ins, how='left', on='SK_ID_CURR')
del ins
gc.collect()

cc = credit_card_balance(col_log=True)
print(f'Group Credit card balance (log) by SK_ID_CURR, shape: {cc.shape}')
df = df.join(cc, how='left', on='SK_ID_CURR')
del cc
gc.collect()

print(f'After {datetime.now()-d1}, join results in Application, shape: {df.shape}')

Concat train samples: 15000, test samples: 15000, shape: (30000, 240)
Join Bureau & Bureau_balance SK_ID_BUREAU, group by SK_ID_CURR, shape: (305811, 116)
Group Previous applications (log) by SK_ID_CURR, shape: (338857, 237)
Group Pos-cash balance by SK_ID_CURR, shape: (337252, 18)
Group Installments payments (log) by SK_ID_CURR, shape: (339587, 22)
Group Credit card balance (log) by SK_ID_CURR, shape: (103558, 141)
After 0:02:02.409022, join results in Application, shape: (30000, 774)


In [12]:
df['TARGET'] = 1 - df['TARGET']
df['TARGET'].fillna(999, inplace=True)
df.fillna(0, inplace=True)

df.to_csv(FILELOG_FNAN0, index=None, sep='\t')
print(f'File saved {FILELOG_FNAN0}, shape: {df.shape}')

del df
gc.collect()

File saved ./credithome_datasets/Credit_Home_Junction_Log_Fnan0_15000.csv, shape: (30000, 774)


16